# **Templates - Making Prompts Dynamic and Reusable**

## **What's Covered?**
1. Introduction to Templates
    - What are Prompt Templates?
    - Why they are crucial?
    - Types of Prompt Templates
    - Input and Output
2. PromptTemplate (for single-string prompts)
    - Creating a PromptTemplate using .from_template()
    - Creating a PromptTemplate by Direct Instantiation
    - Passing values to placeholder using .format()
    - Passing values to placeholder using .format_prompt()
    - Passing values to placeholder using .format_messages()
    - Passing partial_variables
    - Passing partial_variables in an existing template using .partial()
3. The "Templates" vs The "Concrete Messages"
    - Templates
    - Concrete Messages
4. ChatPromptTemplate (for structured chat messages)
    - Creating a ChatPromptTemplate using .from_template()
    - Creating a ChatPromptTemplate using .from_messages()
    - Passing values to placeholder using .format(), .format_messages() and .format_prompt()
    - Creating a ChatPromptTemplate by Direct Instantiation
5. Using LCEL methods
    - .invoke()
    - .pretty_print()
7. MessagesPlaceholder
    - What is MessagesPlaceholder?
    - Regular Placeholder vs MessagePlaceholder
    - How it Works?
    - Making MessagesPlaceholder Optional
    - Limiting the number of messages

## **Introduction to Templates**

### **What are Prompt Templates?**
- Writing **static strings as prompts quickly becomes unmanageable**. We need a way to inject dynamic information. This is where Prompt Template comes in.
- These are the objects that help you **construct prompts dynamically by accepting input variables**. Think of them as blueprints for your prompts.
- Templates **offer a more systematic approach to passing in variables to prompts** for models, instead of using f-string literals or .format() calls. The PromptTemplate converts these into function parameter names that we can pass in.

<img src="images/langchain_model_io.jpg">

### **Why they are crucial?**
1. **Reusability:** Define a template once and use it for many different inputs.
2. **Consistency:** Ensure your prompts follow a specific structure every time, which helps LLMs perform better.
3. **Readability:** Makes your code cleaner by separating prompt logic from other code.
4. **Parameterization:** Easily insert varying information into the prompt.


### **Types of Prompt Templates**
1. **PromptTemplate (for single-string prompts):** Used for generic text completion models or when your prompt is a single string.
2. **ChatPromptTemplate (for structured chat messages):** Specifically designed for chat models, allowing you to define sequences of `HumanMessage`, `AIMessage`, and `SystemMessage` templates. This is generally preferred for modern LLMs as most are fine-tuned for chat.

### **Input and Output**
- We can call a template with **.invoke()** method.
- **Input** to the Prompt Template should be a **dictionary** containing raw user inputs.
- **Output** of a Prompt Template will be a **string** or **list of chat messages**.

## **PromptTemplate (for single-string prompts)**

Used for generic text completion models or when your prompt is a single string.

**Prompt Template**  
- Prompt Templates are used to convert raw user input to a better input to the LLM.
- Templates allow us to easily configure and modify our input prompts to LLM calls.
- A template may include instructions, few-shot examples, and specific context and questions appropriate for a given task.
- LangChain provides tooling to create and work with prompt templates.
- LangChain strives to create model agnostic templates to make it easy to reuse existing templates across different language models.
- Typically, language models expect the prompt to either be a string or else a list of chat messages.

### **Creating a PromptTemplate using .from_template()**

`PromptTemplate.from_template(string_with_placeholders)` helps us create a PromptTemplate object from a string with placeholders (e.g., "Tell me about {topic} in 200 words.").

In [2]:
from langchain_core.prompts import PromptTemplate

# Creating a prompt template with input variables
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

prompt_template

PromptTemplate(input_variables=['adjective', 'content'], input_types={}, partial_variables={}, template='Tell me a {adjective} joke about {content}.')

In [3]:
# Let's check the input variables

prompt_template.input_variables

['adjective', 'content']

### **Creating a PromptTemplate by Direct Instantiation**

You can also instantiate PromptTemplate **directly by passing the template string** and **explicitly listing the input_variables**.

In [71]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(template="Tell me a {adjective} joke about {content}.")

prompt_template

PromptTemplate(input_variables=['adjective', 'content'], input_types={}, partial_variables={}, template='Tell me a {adjective} joke about {content}.')

In [72]:
# Let's check the input variables

prompt_template.input_variables

['adjective', 'content']

### **Passing values to placeholder using .format()**

**.format()** converts the PromptTemplate to a `String`

In [73]:
# format() returns a string

prompt = prompt_template.format(adjective="funny", content="chickens")

print(type(prompt))
print(prompt)

<class 'str'>
Tell me a funny joke about chickens.


### **Passing values to placeholder using .format_prompt()**

**.format_prompt()** converts the PromptTempate to `StringPromptValue`. 

`PromptValues` can be converted to a string and list of messages with the help of `to_string()` and `to_messages()` respectively.

<img src="images/langchain_LCEL.JPG">


In [74]:
# format_prompt() returns a string i.e. StingPromptValue
# PromptValue can be converted to either Strings or Messages

prompt = prompt_template.format_prompt(adjective="funny", content="chickens")

print(type(prompt))
print(prompt)

<class 'langchain_core.prompt_values.StringPromptValue'>
text='Tell me a funny joke about chickens.'


In [75]:
## We can use StingPromptValue and convert it to string using .to_string()

prompt_string = prompt.to_string()

print(type(prompt_string))
print(prompt_string)

<class 'str'>
Tell me a funny joke about chickens.


In [76]:
## We can use StingPromptValue and convert it to List of ChatMessages using .to_messages()

prompt_messages = prompt.to_messages()

print(type(prompt_messages))
print(prompt_messages)

<class 'list'>
[HumanMessage(content='Tell me a funny joke about chickens.', additional_kwargs={}, response_metadata={})]


### **Passing values to placeholder using .format_messages()**

**.format_message()** converts the ChatPromptTemplate to list of `ChatMessages`. **It won't work with the PromptTemplate**.

In [77]:
try:
    prompt = prompt_template.format_messages(adjective="funny", content="chickens")
except:
    print("AttributeError: 'PromptTemplate' object has no attribute 'format_messages'.")

AttributeError: 'PromptTemplate' object has no attribute 'format_messages'.


### **Passing partial_variables**

**partial_variables** in prompt templates are a powerful feature that allows you to "pre-fill" some of the placeholders in your prompt template.

Think of `partial_variables` as **default function parameters**.

**This is particularly useful in scenarios where:**
1. **Some variables are static or known well in advance:** For example, a system instruction that's always the same, or a fixed format_instructions string from an OutputParser. This is covered later in the next chapter (i.e. 4. Output Parsing).
2. **You're building complex chains where some information becomes available earlier:** You can partial a prompt as information becomes available, reducing the number of variables you need to pass around later in the chain.
3. **You need to inject dynamic information from a function:** Instead of a static string, you can provide a function that will be called at the time of formatting to get the value for that variable. This is great for things like current date/time.

In [82]:
from langchain_core.prompts import PromptTemplate

# Method A: Creating a prompt template with input variables
prompt_template = PromptTemplate(
    template="Tell me a {adjective} joke about {content}.",
    partial_variables={"adjective": "funny"}
)

# Method B: Creating a prompt template with input variables
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}.",
    partial_variables={"adjective": "funny"}
)

prompt_template

PromptTemplate(input_variables=['content'], input_types={}, partial_variables={'adjective': 'funny'}, template='Tell me a {adjective} joke about {content}.')

In [83]:
prompt_template.format(content="chickens")

'Tell me a funny joke about chickens.'

In [84]:
prompt_template.format(content="chickens", adjective="dark")

'Tell me a dark joke about chickens.'

### **Passing partial_variables in an existing template using .partial()**

Use the .partial() method on an existing template

In [95]:
from datetime import datetime

# Function to get the current date and time
def get_current_datetime():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

get_current_datetime()

'2025-06-10 18:04:44'

In [101]:
# Prompt template
prompt_template = PromptTemplate.from_template(
    "Today's date is {current_date}. Tell me something interesting about {topic}.",
)

prompt_template

PromptTemplate(input_variables=['current_date', 'topic'], input_types={}, partial_variables={}, template="Today's date is {current_date}. Tell me something interesting about {topic}.")

In [110]:
# Declaring date as a partial variable using a function
# NOTE: passing the function without using paranthesis will help in dynamically generating 'current_date'

date_prompt = prompt_template.partial(current_date=get_current_datetime)

date_prompt

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={'current_date': <function get_current_datetime at 0x114680790>}, template="Today's date is {current_date}. Tell me something interesting about {topic}.")

In [111]:
# Invoke the prompt
formatted_prompt = date_prompt.format(topic="outer space")
print(formatted_prompt)

Today's date is 2025-06-10 18:11:43. Tell me something interesting about outer space.


In [112]:
# Run it again, the date will update
formatted_prompt_again = date_prompt.format(topic="oceans")
print(formatted_prompt_again)

Today's date is 2025-06-10 18:11:50. Tell me something interesting about oceans.


## **The "Templates" vs The "Concrete Messages"**

`*MessagePromptTemplate` is like a form with empty fields (e.g., "Customer Name: ______, Product: ______"). You define the fields and their labels.

`*Message` is like a filled-out form (e.g., "Customer Name: John Doe, Product: Laptop"). It has all the information specific to that one instance.

### **Templates**
These are classes that represent templates for specific types of messages (System, Human, AI). They define the structure and content of a message, often including placeholders ({variable_name}) that need to be filled in later. Example:
- SystemMessagePromptTemplate
- HumanMessagePromptTemplate
- AIMessagePromptTemplate

They are not directly sent to the LLM. They first need to be "formatted" or "invoked" to produce concrete `*Message` objects.

In [121]:
from langchain_core.prompts import SystemMessagePromptTemplate

system_template = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant specialized in {topic}."
)

system_template

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='You are a helpful assistant specialized in {topic}.'), additional_kwargs={})

In [132]:
from langchain_core.prompts import HumanMessagePromptTemplate

human_template = HumanMessagePromptTemplate.from_template(
    "My name is {user_name}. Can you explain {concept}?", 
    partial_variables={"user_name":"ThatAIGuy"}
)

human_template

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['concept'], input_types={}, partial_variables={'user_name': 'ThatAIGuy'}, template='My name is {user_name}. Can you explain {concept}?'), additional_kwargs={})

### **Concrete Messages**
These are actual, concrete message objects that contain the final, filled-in text content and their respective roles (system, human, AI). They represent a single turn or instruction in a conversation. Example:
- SystemMessage
- HumanMessage
- AIMessage

They are the "data" that the LLM processes.

They do not contain placeholders ({}). Their content is fully resolved.

In [129]:
from langchain_core.messages import SystemMessage

system_instruction = SystemMessage(content="You are a polite and friendly chatbot.")

system_instruction

SystemMessage(content='You are a polite and friendly chatbot.', additional_kwargs={}, response_metadata={})

In [130]:
from langchain_core.messages import HumanMessage

user_query = HumanMessage(content="What is the weather like today?")

user_query

HumanMessage(content='What is the weather like today?', additional_kwargs={}, response_metadata={})

In [131]:
from langchain_core.messages import AIMessage

ai_response = AIMessage(content="I'm sorry, I don't have access to real-time weather information.")

ai_response

AIMessage(content="I'm sorry, I don't have access to real-time weather information.", additional_kwargs={}, response_metadata={})

## **ChatPromptTemplate (for structured chat messages)**

This class is the preferred way to create prompts for modern chat-based LLMs. 

You provide a list of message "templates," where each template represents a turn in the conversation and can have a specific role (system, human, ai) and placeholders.

### **Creating a ChatPromptTemplate using .from_template()**

Even though this method is available for ChatPromptTemplate, but it's important to understand its limitation: it creates a single human message template. It's useful if you only have one dynamic part from the user.

In [113]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template(
    "Tell me about {topic_name}?"
)

In [114]:
chat_template.input_variables

['topic_name']

### **Creating a ChatPromptTemplate using .from_messages()**

This is the most versatile and recommended way to create a ChatPromptTemplate. You provide a list of message "templates," where each template represents a turn in the conversation and can have a specific role (system, human, ai) and placeholders.

In [115]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage

# Option A: Using message objects explicitly
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("You are a helpful AI bot. Your name is {bot_name}.", 
                                                  partial_variables={"bot_name": "ThatAIGuy"}),
        HumanMessage(content="Hello, how are you doing?"),
        AIMessage(content="I'm doing well, thanks!"),
        HumanMessagePromptTemplate.from_template("Tell me about {topic_name}."),
    ]
)

In [116]:
chat_template.input_variables

['topic_name']

In [117]:
from langchain_core.prompts import ChatPromptTemplate

# Option B: Using tuples (shorthand) - (role_string, template_string)
# This is often more concise and very common.
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {bot_name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "Tell me about {topic_name}."),
    ]
)

In [93]:
partial_chat_template = chat_template.partial(bot_name="ThatAIGuy")

partial_chat_template

ChatPromptTemplate(input_variables=['topic_name'], input_types={}, partial_variables={'bot_name': 'ThatAIGuy'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['bot_name'], input_types={}, partial_variables={}, template='You are a helpful AI bot. Your name is {bot_name}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, how are you doing?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="I'm doing well, thanks!"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic_name'], input_types={}, partial_variables={}, template='Tell me about {topic_name}.'), additional_kwargs={})])

In [94]:
partial_chat_template.input_variables

['topic_name']

### **Passing values to placeholder using .format(), .format_messages() and .format_prompt()**

Templates offer a more systematic approach to passing in variables to prompts for models, instead of using f-string literals or .format() calls. The PromptTemplate converts these into function parameter names that we can pass in.

- .format(): Converts the PromptTemplate to `String`
- .format_prompt(): Converts the PromptTempate to `ChatPromptValue`. `PromptValues` can be converted to both LLM (to string) inputs and ChatModel (to messages) inputs. On this we can apply `to_messages()` or `to_string()`.
- .format_message(): Converts the PromptTemplate to list of `ChatMessages`

In [38]:
chat_template.format(bot_name="Bob", topic_name="LangChain")

"System: You are a helpful AI bot. Your name is Bob.\nHuman: Hello, how are you doing?\nAI: I'm doing well, thanks!\nHuman: Tell me about LangChain."

In [39]:
# format_prompt() returns a ChatPromptValue()

chat_template.format_prompt(bot_name="Bob", topic_name="LangChain")

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about LangChain.', additional_kwargs={}, response_metadata={})])

In [40]:
# format_messages() returns chat messages 

chat_template.format_messages(bot_name="Bob", topic_name="LangChain")

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Tell me about LangChain.', additional_kwargs={}, response_metadata={})]

### **Creating a ChatPromptTemplate by Direct Instantiation**

In [133]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate(
    messages = [
        ("system", "You are a helpful AI bot. Your name is {bot_name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "Tell me about {topic_name}."),
    ],
    partial_variables={"bot_name": "Alice"}
)

chat_template

ChatPromptTemplate(input_variables=['topic_name'], input_types={}, partial_variables={'bot_name': 'Alice'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['bot_name'], input_types={}, partial_variables={}, template='You are a helpful AI bot. Your name is {bot_name}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, how are you doing?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="I'm doing well, thanks!"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic_name'], input_types={}, partial_variables={}, template='Tell me about {topic_name}.'), additional_kwargs={})])

In [134]:
# format() returns a string

prompt = chat_template.format(topic_name="machine learning")

print(type(prompt))
print(prompt)

<class 'str'>
System: You are a helpful AI bot. Your name is Alice.
Human: Hello, how are you doing?
AI: I'm doing well, thanks!
Human: Tell me about machine learning.


In [135]:
# format_prompt() returns a chat here i.e. ChatPromptValue()
# PromptValue can be converted to either Strings or Chat Messages

prompt = chat_template.format_prompt(topic_name="machine learning")

print(type(prompt))
print(prompt)

<class 'langchain_core.prompt_values.ChatPromptValue'>
messages=[SystemMessage(content='You are a helpful AI bot. Your name is Alice.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about machine learning.', additional_kwargs={}, response_metadata={})]


In [136]:
# fomat_messages() return list of chat messages

prompt = chat_template.format_messages(topic_name="machine learning")

print(type(prompt))
print(prompt)

<class 'list'>
[SystemMessage(content='You are a helpful AI bot. Your name is Alice.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about machine learning.', additional_kwargs={}, response_metadata={})]


## **Using LCEL methods**

### **.invoke()**

`PromptTemplate` and `ChatPromptTemplate` implement the Runnable interface, the basic building block of the **LangChain Expression Language (LCEL)**. This means they support `invoke`, `ainvoke`, `stream`, `astream`, `batch`, `abatch`, `astream_log` calls.

**Using `invoke()`:**  
`PromptTemplate` **accepts a dictionary (of the prompt variables)** and returns a `StringPromptValue`.  

A `ChatPromptTemplate` **accepts a dictionary (of the prompt variables)** and returns a `ChatPromptValue`.

In [137]:
chat_template.invoke({"topic_name": "LangChain"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Alice.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about LangChain.', additional_kwargs={}, response_metadata={})])

In [138]:
print(chat_template.invoke({"topic_name": "LangChain"}).to_string())

System: You are a helpful AI bot. Your name is Alice.
Human: Hello, how are you doing?
AI: I'm doing well, thanks!
Human: Tell me about LangChain.


In [139]:
print(chat_template.invoke({"topic_name": "LangChain"}).to_messages())

[SystemMessage(content='You are a helpful AI bot. Your name is Alice.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about LangChain.', additional_kwargs={}, response_metadata={})]


### **.pretty_print()**

The purpose of `.pretty_print()` is to display a human-readable, graphical representation of your chain's structure and flow.

In [140]:
chat_template.pretty_print()

================================ System Message ================================

You are a helpful AI bot. Your name is {bot_name}.

================================ Human Message =================================

Hello, how are you doing?

================================== AI Message ==================================

I'm doing well, thanks!

================================ Human Message =================================

Tell me about {topic_name}.


In [65]:
for msg in chat_template.invoke({"topic_name": "LangChain"}).to_messages():
    msg.pretty_print()

================================ System Message ================================

You are a helpful AI bot. Your name is Alice.
================================ Human Message =================================

Hello, how are you doing?
================================== Ai Message ==================================

I'm doing well, thanks!
================================ Human Message =================================

Tell me about LangChain.


## **MessagesPlaceholder**

### **What is MessagesPlaceholder?**
At its core, `MessagesPlaceholder` is a special type of "placeholder" within a `ChatPromptTemplate` that is designed to accept a list of `BaseMessage` objects.

`MessagesPlaceholder` provides the exact slot in your `ChatPromptTemplate` where this evolving list of messages can be inserted.

### **Regular Placeholder vs MessagePlaceholder**
Unlike a regular placeholder like {variable_name} which expects a single string or simple value, `MessagesPlaceholder` expects a sequence of messages (like HumanMessage, AIMessage, SystemMessage).

### **How it Works?**

When you include MessagesPlaceholder in your ChatPromptTemplate.from_messages() or ChatPromptTemplate() definition:
1. You give it a variable_name (e.g., "chat_history").
2. When you invoke() the ChatPromptTemplate (or a chain containing it), you must pass a key in your input dictionary that matches this variable_name. The value for this key must be a list of BaseMessage objects.
3. LangChain then takes this list of messages and inserts them directly into the position specified by MessagesPlaceholder within the final list of messages sent to the LLM.

In [165]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Simulate some pre-existing chat history
current_chat_history = [
    HumanMessage(content="What's your favorite color?"),
    AIMessage(content="As an AI, I don't have feelings or preferences, so I don't have a favorite color."),
    HumanMessage(content="Oh, I see. What's your favorite animal then?"),
    AIMessage(content="Similarly, I don't have personal experiences to develop preferences for animals."),
]

In [166]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate

# Define a ChatPromptTemplate that includes MessagesPlaceholder
chat_prompt_with_history = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful and knowledgeable assistant."),
        # This is where the magic happens: insert the chat history
        MessagesPlaceholder(variable_name="chat_history"),
        # The new human message for the current turn
        HumanMessagePromptTemplate.from_template("{new_question}"),
    ]
)

chat_prompt_with_history.pretty_print()

================================ System Message ================================

You are a helpful and knowledgeable assistant.

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{new_question}


In [167]:
# When invoking, pass the actual chat_history list and the new question

formatted_messages = chat_prompt_with_history.invoke({
    "chat_history": current_chat_history,
    "new_question": "Can you summarize our conversation so far?"
})

formatted_messages

ChatPromptValue(messages=[SystemMessage(content='You are a helpful and knowledgeable assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's your favorite color?", additional_kwargs={}, response_metadata={}), AIMessage(content="As an AI, I don't have feelings or preferences, so I don't have a favorite color.", additional_kwargs={}, response_metadata={}), HumanMessage(content="Oh, I see. What's your favorite animal then?", additional_kwargs={}, response_metadata={}), AIMessage(content="Similarly, I don't have personal experiences to develop preferences for animals.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you summarize our conversation so far?', additional_kwargs={}, response_metadata={})])

In [168]:
for msg in formatted_messages.to_messages():
    msg.pretty_print()

================================ System Message ================================

You are a helpful and knowledgeable assistant.
================================ Human Message =================================

What's your favorite color?
================================== Ai Message ==================================

As an AI, I don't have feelings or preferences, so I don't have a favorite color.
================================ Human Message =================================

Oh, I see. What's your favorite animal then?
================================== Ai Message ==================================

Similarly, I don't have personal experiences to develop preferences for animals.
================================ Human Message =================================

Can you summarize our conversation so far?


### **Making MessagesPlaceholder Optional**

In [169]:
user_input = {"new_question": "Hi, My name is ThatAIGuy."}

try:
    chat_prompt_with_history.invoke(user_input)
except:
    print("KeyError: Input to ChatPromptTemplate is missing variables {'chat_history'}.")

KeyError: Input to ChatPromptTemplate is missing variables {'chat_history'}.


In [170]:
# Define a ChatPromptTemplate that includes MessagesPlaceholder
chat_prompt_with_history = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful and knowledgeable assistant."),
        # This is where the magic happens: insert the chat history
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        # The new human message for the current turn
        HumanMessagePromptTemplate.from_template("{new_question}"),
    ]
)

chat_prompt_with_history.pretty_print()

================================ System Message ================================

You are a helpful and knowledgeable assistant.

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{new_question}


In [171]:
user_input = {"new_question": "Hi, My name is ThatAIGuy."}

for msg in chat_prompt_with_history.invoke(user_input).to_messages():
    msg.pretty_print()

================================ System Message ================================

You are a helpful and knowledgeable assistant.
================================ Human Message =================================

Hi, My name is ThatAIGuy.


### **Limiting the number of messages**

In [181]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Simulate some pre-existing chat history
current_chat_history = [
    HumanMessage(content="What's your favorite color?"),
    AIMessage(content="As an AI, I don't have feelings or preferences, so I don't have a favorite color."),
    HumanMessage(content="Oh, I see. What's your favorite animal then?"),
    AIMessage(content="Similarly, I don't have personal experiences to develop preferences for animals."),
]

In [188]:
# Define a ChatPromptTemplate that includes MessagesPlaceholder
chat_prompt_with_history = ChatPromptTemplate(
    messages = [ SystemMessage(content="You are a helpful and knowledgeable assistant."),
                # This is where the magic happens: insert the chat history
                MessagesPlaceholder(variable_name="chat_history", optional=True, n_messages=2),
                # The new human message for the current turn
                HumanMessagePromptTemplate.from_template("{new_question}"),
                ] 
)

chat_prompt_with_history.pretty_print()

================================ System Message ================================

You are a helpful and knowledgeable assistant.

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{new_question}


In [189]:
# When invoking, pass the actual chat_history list and the new question

formatted_messages = chat_prompt_with_history.invoke({
    "chat_history": current_chat_history,
    "new_question": "Can you summarize our conversation so far?"
})

for msg in formatted_messages.to_messages():
    msg.pretty_print()

================================ System Message ================================

You are a helpful and knowledgeable assistant.
================================ Human Message =================================

Oh, I see. What's your favorite animal then?
================================== Ai Message ==================================

Similarly, I don't have personal experiences to develop preferences for animals.
================================ Human Message =================================

Can you summarize our conversation so far?
